In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# loading forage quality data
import math
import re
import numpy as np
import pandas as pd

tmean = pd.read_csv('/Users/edwardamoah/Documents/GitHub/OsmiaPopModel/output/tmean_prism_pennsylvania_data_1990_2023.csv')
ppt = pd.read_csv('/Users/edwardamoah/Documents/GitHub/OsmiaPopModel/output/ppt_prism_pennsylvania_data_1990_2023.csv')
#forage = pd.read_csv('/Users/edwardamoah/Documents/GitHub/OsmiaPopModel/output/foraging_quality_pennsylvania_data.csv')
forage = pd.read_csv('/Users/edwardamoah/Documents/GitHub/OsmiaPopModel/data/forage.csv')

cols = tmean.columns.tolist()

# create new columns 
new_cols = []
for col in cols:
    if "Forage" in col:
        code = col.split("_")[1]
        number = re.findall(r'\d+', code)[0]
        new_cols.append(number)
    else:
        new_cols.append(col)

new_cols

# rename columns
tmean = tmean.rename(columns=dict(zip(tmean.columns, new_cols)))

# create grid_id column
tmean["grid_id"] = tmean["col"].astype(str) + "_" + tmean["row"].astype(str)

possible_cols = tmean.col.tolist()
possible_rows = tmean.row.tolist()

def get_adjacent_cols_rows(col, row, possible_cols=possible_cols, possible_rows=possible_rows):
    adjacent_cols_rows = []
    
    for c, r in zip(possible_cols, possible_rows):
        if abs(c - col) <= 1 and abs(r - row) <= 1 and (c != col or r != row):
            adjacent_cols_rows.append((c, r))
    
    return adjacent_cols_rows

forage["grid_id"] = forage["col"].astype(str) + "_" + forage["row"].astype(str)

def getGridForageQuality(col, row, year, forage=forage):
    grid_id = str(col) + "_" + str(row)
    try:
        return forage.query(f" grid_id == '{grid_id}' and year == {year}")['Forage_spring_1km'].values[0]
    except:
        return getGridForageQuality(col, row, year + 1)

'''
def getGridForageQuality(col, row, year):

    grid_quality = getGridQuality(col, row, year)
    if math.isnan(grid_quality):
        adjacent_cols_rows = get_adjacent_cols_rows(col, row)
        adjacent_quality = []
        for c, r in adjacent_cols_rows:
            grid_quality = getGridQuality(c, r, year)
            if not math.isnan(grid_quality):
                adjacent_quality.append(grid_quality)
        return np.mean(adjacent_quality)
    else:
        return grid_quality
'''



################# tmean data #################

cols = tmean.columns.tolist()

new_cols = []
for col in cols:
    if "PRISM_tmean" in col:
        new_cols.append(col.split("_")[-2])
    else:
        new_cols.append(col)
new_cols

tmean = tmean.rename(columns=dict(zip(tmean.columns, new_cols)))


tmean["grid_id"] = tmean["col"].astype(str) + "_" + tmean["row"].astype(str)

def getTmean(col, row, day, month, year, tmean=tmean):
    '''
    Get temperature data for a given grid cell
    col: int, column number
    row: int, row number
    day: int, day of the month
    month: int, month of the year
    year: int, year
    tmean: pd.DataFrame, temperature data

    output:
    float, temperature in degrees celcius
    '''
    if month < 10:
        month = "0" + str(month)
    if day < 10:
        day = "0" + str(day)
    date = str(year) + "" + str(month) + "" + str(day)
    grid_id = str(col) + "_" + str(row)
    tmean = tmean[tmean["grid_id"] == grid_id][date].values[0]
    return tmean

############### ppt data #################

cols = ppt.columns.tolist()

new_cols = []
for col in cols:
    if "PRISM_ppt" in col:
        new_cols.append(col.split("_")[-2])
    else:
        new_cols.append(col)

ppt = ppt.rename(columns=dict(zip(ppt.columns, new_cols)))

ppt["grid_id"] = ppt["col"].astype(str) + "_" + ppt["row"].astype(str)

def getPpt(col, row, day, month, year, ppt=ppt):
    '''
    Get precipitation data for a given grid cell
    col: int, column number
    row: int, row number
    day: int, day of the month
    month: int, month of the year
    year: int, year
    ppt: pd.DataFrame, precipitation data

    output:
    float, precipitation in mm
    '''
    if month < 10:
        month = "0" + str(month)
    if day < 10:
        day = "0" + str(day)
    date = str(year) + "" + str(month) + "" + str(day)
    grid_id = str(col) + "_" + str(row)
    ppt = ppt[ppt["grid_id"] == grid_id][date].values[0]
    return ppt

In [3]:
################ Modeling ################

In [3]:
grid_id = '1092_242'
year = 2009
forage.query(f" grid_id == '{grid_id}' and year == {year}")['Forage_spring_1km'].values[0]

0.61875

In [29]:
grids = pd.read_csv('/Users/edwardamoah/Documents/Dissertation/Osmia_Pop_Simulation/pennsylvania_prism_grid_points_centre_county.csv')

In [30]:
grids = tmean[["col", "row", "grid_id"]]
grids

,col,row,grid_id
0,1083,184,1083_184
1,1084,184,1084_184
2,1085,184,1085_184
3,1086,184,1086_184
4,1081,185,1081_185
...,...,...,...
7447,1178,245,1178_245
7448,1179,245,1179_245
7449,1180,245,1180_245
7450,1181,245,1181_245


In [5]:
# Emergence Model
import datetime
from datetime import datetime, timedelta

emergence_dict = {}

def getEmergence(col, row, year):
    '''
    Get emergence date for a given grid cell
    col: int, column number
    row: int, row number
    year: int, year

    output:
    datetime.date, emergence date
    '''

    dict_key = str(col) + "_" + str(row) + "_" + str(year)
    if dict_key in emergence_dict:
        return emergence_dict[dict_key]

    # function constants
    thermal_temp = 209 # adam et al
    dev_temp = 6.53 # adam et al


    starting_date = f"{year-1}-11-01" # November 1st of the previous year
    starting_date = datetime.strptime(starting_date, "%Y-%m-%d").date()
    cumulative_degrees = 0.0
    new_date = starting_date


    # determine emergence date
    cumulative_degrees = 0.0
    while cumulative_degrees < thermal_temp:
        daily_tmean = getTmean(col, row, new_date.day, new_date.month, new_date.year)
        if daily_tmean >= dev_temp:
            cumulative_degrees += float(daily_tmean - dev_temp)
            

        new_date += timedelta(days=1)

    emergence_dict[dict_key] = new_date

    return new_date

In [6]:
year = 2023
starting_date = f"{year-1}-11-01" # November 1st of the previous year
starting_date = datetime.strptime(starting_date, "%Y-%m-%d").date()
starting_date.strftime("%Y-%m-%d")

'2022-11-01'

In [7]:
# Reproduction Model

precipitation_threshold = 1

reproductive_dict = {}

def getSpatiallyExplicitReproduction(col, row, emergence, longevity=22, temperature_threshold=13.9, mating_days=2, precipitation_threshold=5, forage_threshold=0.5):
    '''
    Get reproduction date for a given grid cell
    col: int, column number
    row: int, row number
    year: int, year
    emergence: datetime.date, emergence date

    output:
    datetime.date, reproduction date
    '''

    dict_key = str(col) + "_" + str(row) + "_" + emergence.strftime("%Y-%m-%d")
    if dict_key in reproductive_dict:
        return reproductive_dict[dict_key]

    # function constants
    eggs = 0
    egg_delta = 1 # change in eggs per day
     # update starting date
    starting_date = emergence + timedelta(days=mating_days)

    # update egg_delta based on foraging quality
    forage_quality = getGridForageQuality(col, row, starting_date.year)
    if forage_quality < forage_threshold:
        egg_delta = 1
    else:
        egg_delta = 2

    non_foraging_days = 0
    precipitation_effect = 0
    temperature_effect = 0
    temps = []
    ppts = []
    egg_days = []
    
    for i in range(longevity-mating_days):
        # get daily temperature and precipitation
        daily_tmean = getTmean(col, row, starting_date.day, starting_date.month, starting_date.year)
        daily_ppt = getPpt(col, row, starting_date.day, starting_date.month, starting_date.year)

        temps.append(daily_tmean)
        ppts.append(daily_ppt)

        # update eggs
        if daily_tmean >= temperature_threshold and daily_ppt < precipitation_threshold:
            eggs += egg_delta
            egg_days.append(starting_date.strftime("%Y-%m-%d"))
            
        else:
            non_foraging_days += 1
            if daily_tmean < temperature_threshold:
                temperature_effect += 1
            if daily_ppt >= precipitation_threshold:
                precipitation_effect += 1

        # update date
        starting_date += timedelta(days=1)
        

    return_dict = {
        "eggs": eggs,
        "non_foraging_days": non_foraging_days,
        "precipitation_effect": precipitation_effect,
        "temperature_effect": temperature_effect,
        "temps": temps,
        "ppts": ppts,
        "egg_days": egg_days,
        'forage_quality': forage_quality
    }

    reproductive_dict[dict_key] = return_dict

    return return_dict

In [8]:
# Egg and Larva Mortality Model

def getEggLarvaMortality(col, row, emergence, reproduction, mating_days=2):

    def getEggMortalityProbability(date, tmeans, mortality_delta = 0.1):
        # constants
        ldt = 10 # c
        udt = 30 # c

        prob = 0
        cold_days = 0
        hot_days = 0
        for i in range(18): # 4 egg + 14 larva
            if tmeans[date] < ldt or tmeans[date] > udt:
                prob += mortality_delta
                if tmeans[date] < ldt:
                    cold_days += 1
                if tmeans[date] > udt:
                    hot_days += 1
            date = datetime.strptime(date, "%Y-%m-%d").date()
            date += timedelta(days=1)
            date = date.strftime("%Y-%m-%d")

            

        return {
            "prob": prob,
            "cold_days": cold_days,
            "hot_days": hot_days
        }

    total_dev_days = 38 # days 20 forages + 4 egg + 14 larva

    startdate = emergence + timedelta(days=mating_days)

    tmeans = {}
    for i in range(total_dev_days): # get all the tmeans

        tmeans[startdate.strftime("%Y-%m-%d")] = getTmean(col, row, startdate.day, startdate.month, startdate.year)
        startdate += timedelta(days=1)

    egg_mortalities = []
    cold_days = []
    hot_days = []
    for egg_day in reproduction['egg_days']:
        mortality = getEggMortalityProbability(egg_day, tmeans)
        egg_mortalities.append(mortality['prob'])
        cold_days.append(mortality['cold_days'])
        hot_days.append(mortality['hot_days'])

    if len(egg_mortalities) == 0:
        return {
            "mortality": 0,
            "cold_days": 0,
            "hot_days": 0
        }

    return {    
        "mortality": np.mean(egg_mortalities),
        "cold_days": np.mean(cold_days),
        "hot_days": np.mean(hot_days)
    }


In [9]:
# Pre-wintering Mortality Model

def getPreWinteringMortality(col, row, year, pre_winter_delta=0.0025):

    startdate = datetime(year, 9, 1)
    enddate = datetime(year, 11, 1)

    dev_temp = 6.53

    mortality = 0
    dev_days = 0
    cumulative_degrees = 0
    
    while startdate < enddate:
        daily_tmean = getTmean(col, row, startdate.day, startdate.month, startdate.year)
        if daily_tmean > dev_temp:
            mortality += pre_winter_delta
            dev_days += 1
            cumulative_degrees += float(daily_tmean - dev_temp)

        startdate += timedelta(days=1)

    return {
        "mortality": mortality,
        "dev_days": dev_days,
        "cumulative_degrees": cumulative_degrees
    }
    

In [10]:
# Wintering Mortality Model

# Wintering Mortality Model

def getWinteringTemperature(col, row, year):
        
        dev_temp = 6.53
    
        startdate = datetime(year, 11, 1)
        #enddate = datetime(year+1, 2, 1)
        try:
            enddate = getEmergence(col, row, year+1)
            enddate = datetime(enddate.year, enddate.month, enddate.day)
        except:
            enddate = datetime(year+1, 2, 1)

    
        temps = []
    
        while startdate < enddate:
            daily_tmean = getTmean(col, row, startdate.day, startdate.month, startdate.year)
            temps.append(daily_tmean)
    
            startdate += timedelta(days=1)
        
        winter_temp = np.mean(temps)

        if winter_temp > dev_temp:
            return {
                "winter_temp": winter_temp,
                "mortality": 0.5
            }
        else:
            return {
                "winter_temp": winter_temp,
                "mortality": 0
            }

In [11]:
startdate = datetime(year, 11, 1)
startdate

datetime.datetime(2023, 11, 1, 0, 0)

In [12]:
######################## Run Simulation ################

In [32]:
#years = [2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]
#years = [2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]
years = [1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]
years = [2020, 2021, 2022, 2023]
years = [2016, 2017, 2018, 2019]

In [14]:
col = 1131
row = 219
df = pd.DataFrame(years, columns=["year"])
df['emergence'] = df['year'].apply(lambda x: getEmergence(col, row, x))
df['reproduction'] = df.apply(lambda x: getSpatiallyExplicitReproduction(col, row, x['emergence']), axis=1)
df['egg_larva_mortality'] = df.apply(lambda x: getEggLarvaMortality(col, row, x['emergence'], x['reproduction']), axis=1)
df['pre_winter_mortality'] = df['year'].apply(lambda x: getPreWinteringMortality(col, row, x))
df['winter_mortality'] = df['year'].apply(lambda x: getWinteringTemperature(col, row, x))


In [17]:
df['winter_mortality'].tolist()

[{'winter_temp': 2.7877408463660394, 'mortality': 0},
 {'winter_temp': 2.649619408636481, 'mortality': 0},
 {'winter_temp': 3.60078907100882, 'mortality': 0},
 {'winter_temp': 2.5706118755852403, 'mortality': 0}]

In [28]:
grids

,col,row,grid_id,emergence_2020,reproduction_2020,egg_larva_mortality_2020,pre_winter_mortality_2020,winter_mortality_2020,emergence_2021,reproduction_2021,...,emergence_2022,reproduction_2022,egg_larva_mortality_2022,pre_winter_mortality_2022,winter_mortality_2022,emergence_2023,reproduction_2023,egg_larva_mortality_2023,pre_winter_mortality_2023,winter_mortality_2023
0,1083,184,1083_184,2020-05-26,"{'eggs': 13, 'non_foraging_days': 7, 'precipit...","{'egg_mortality': 0.007692307692307693, 'cold_...","{'mortality': 0.15000000000000008, 'dev_days':...","{'winter_temp': 2.7354601948111332, 'mortality...",2021-04-11,"{'eggs': 1, 'non_foraging_days': 19, 'precipit...",...,2022-05-12,"{'eggs': 11, 'non_foraging_days': 9, 'precipit...","{'egg_mortality': 0.02727272727272728, 'cold_d...","{'mortality': 0.14000000000000007, 'dev_days':...","{'winter_temp': 3.3761900672258416, 'mortality...",2023-04-23,"{'eggs': 4, 'non_foraging_days': 16, 'precipit...","{'egg_mortality': 0.22500000000000003, 'cold_d...","{'mortality': 0.14750000000000008, 'dev_days':...","{'winter_temp': 3.4156933742696824, 'mortality..."
1,1084,184,1084_184,2020-05-26,"{'eggs': 13, 'non_foraging_days': 7, 'precipit...","{'egg_mortality': 0.007692307692307693, 'cold_...","{'mortality': 0.15000000000000008, 'dev_days':...","{'winter_temp': 2.710003038671631, 'mortality'...",2021-04-11,"{'eggs': 1, 'non_foraging_days': 19, 'precipit...",...,2022-05-12,"{'eggs': 11, 'non_foraging_days': 9, 'precipit...","{'egg_mortality': 0.02727272727272728, 'cold_d...","{'mortality': 0.14000000000000007, 'dev_days':...","{'winter_temp': 3.4424062657433785, 'mortality...",2023-04-23,"{'eggs': 4, 'non_foraging_days': 16, 'precipit...","{'egg_mortality': 0.22500000000000003, 'cold_d...","{'mortality': 0.14750000000000008, 'dev_days':...","{'winter_temp': 3.3847727777193417, 'mortality..."
2,1085,184,1085_184,2020-05-24,"{'eggs': 13, 'non_foraging_days': 7, 'precipit...","{'egg_mortality': 0.0, 'cold_days': 0.0, 'hot_...","{'mortality': 0.15000000000000008, 'dev_days':...","{'winter_temp': 2.674645267959152, 'mortality'...",2021-04-11,"{'eggs': 1, 'non_foraging_days': 19, 'precipit...",...,2022-05-11,"{'eggs': 12, 'non_foraging_days': 8, 'precipit...","{'egg_mortality': 0.03333333333333333, 'cold_d...","{'mortality': 0.14000000000000007, 'dev_days':...","{'winter_temp': 3.4608536882681302, 'mortality...",2023-04-23,"{'eggs': 6, 'non_foraging_days': 14, 'precipit...","{'egg_mortality': 0.21666666666666667, 'cold_d...","{'mortality': 0.14750000000000008, 'dev_days':...","{'winter_temp': 3.2691770823793895, 'mortality..."
3,1086,184,1086_184,2020-05-23,"{'eggs': 14, 'non_foraging_days': 6, 'precipit...","{'egg_mortality': 0.0, 'cold_days': 0.0, 'hot_...","{'mortality': 0.14500000000000007, 'dev_days':...","{'winter_temp': 2.557405505121123, 'mortality'...",2021-04-11,"{'eggs': 1, 'non_foraging_days': 19, 'precipit...",...,2022-05-11,"{'eggs': 13, 'non_foraging_days': 7, 'precipit...","{'egg_mortality': 0.03076923076923077, 'cold_d...","{'mortality': 0.14000000000000007, 'dev_days':...","{'winter_temp': 3.405440408429614, 'mortality'...",2023-04-23,"{'eggs': 6, 'non_foraging_days': 14, 'precipit...","{'egg_mortality': 0.16666666666666666, 'cold_d...","{'mortality': 0.14750000000000008, 'dev_days':...","{'winter_temp': 3.0572748937360616, 'mortality..."
4,1081,185,1081_185,2020-05-25,"{'eggs': 13, 'non_foraging_days': 7, 'precipit...","{'egg_mortality': 0.015384615384615385, 'cold_...","{'mortality': 0.15000000000000008, 'dev_days':...","{'winter_temp': 2.745083035388962, 'mortality'...",2021-04-10,"{'eggs': 1, 'non_foraging_days': 19, 'precipit...",...,2022-05-11,"{'eggs': 10, 'non_foraging_days': 10, 'precipi...","{'egg_mortality': 0.04, 'cold_days': 0.4, 'hot...","{'mortality': 0.14000000000000007, 'dev_days':...","{'winter_temp': 3.327987202028662, 'mortality'...",2023-04-23,"{'eggs': 5, 'non_foraging_days': 15, 'precipit...","{'egg_mortality': 0.24000000000000005, 'cold_d...","{'mortality

In [33]:
for year in years:
    grids[f"emergence_{year}"] = grids.apply(lambda x: getEmergence(x["col"], x["row"], year), axis=1)
    grids[f"reproduction_{year}"] = grids.apply(lambda x: getSpatiallyExplicitReproduction(x["col"], x["row"], x[f"emergence_{year}"]), axis=1)
    grids[f"egg_larva_mortality_{year}"] = grids.apply(lambda x: getEggLarvaMortality(x["col"], x["row"], x[f"emergence_{year}"], x[f"reproduction_{year}"]), axis=1)
    grids[f"pre_winter_mortality_{year}"] = grids.apply(lambda x: getPreWinteringMortality(x["col"], x["row"], year), axis=1)
    grids[f"winter_mortality_{year}"] = grids.apply(lambda x: getWinteringTemperature(x["col"], x["row"], year), axis=1)

/var/folders/r5/48dgr3g12qg3cwmg9f6v0cwc0000gn/T/ipykernel_40894/1403631818.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grids[f"emergence_{year}"] = grids.apply(lambda x: getEmergence(x["col"], x["row"], year), axis=1)
/var/folders/r5/48dgr3g12qg3cwmg9f6v0cwc0000gn/T/ipykernel_40894/1403631818.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grids[f"reproduction_{year}"] = grids.apply(lambda x: getSpatiallyExplicitReproduction(x["col"], x["row"], x[f"emergence_{year}"]), axis=1)
/Users/edwardamoa

KeyboardInterrupt: 

In [24]:
grids

,col,row,grid_id,emergence_2020,reproduction_2020,egg_larva_mortality_2020,pre_winter_mortality_2020,winter_mortality_2020,emergence_2021,reproduction_2021,...,emergence_2022,reproduction_2022,egg_larva_mortality_2022,pre_winter_mortality_2022,winter_mortality_2022,emergence_2023,reproduction_2023,egg_larva_mortality_2023,pre_winter_mortality_2023,winter_mortality_2023
0,1083,184,1083_184,2020-05-26,"{'eggs': 13, 'non_foraging_days': 7, 'precipit...","{'egg_mortality': 0.007692307692307693, 'cold_...","{'mortality': 0.15000000000000008, 'dev_days':...","{'winter_temp': 2.7354601948111332, 'mortality...",2021-04-11,"{'eggs': 1, 'non_foraging_days': 19, 'precipit...",...,2022-05-12,"{'eggs': 11, 'non_foraging_days': 9, 'precipit...","{'egg_mortality': 0.02727272727272728, 'cold_d...","{'mortality': 0.14000000000000007, 'dev_days':...","{'winter_temp': 3.3761900672258416, 'mortality...",2023-04-23,"{'eggs': 4, 'non_foraging_days': 16, 'precipit...","{'egg_mortality': 0.22500000000000003, 'cold_d...","{'mortality': 0.14750000000000008, 'dev_days':...","{'winter_temp': 3.4156933742696824, 'mortality..."
1,1084,184,1084_184,2020-05-26,"{'eggs': 13, 'non_foraging_days': 7, 'precipit...","{'egg_mortality': 0.007692307692307693, 'cold_...","{'mortality': 0.15000000000000008, 'dev_days':...","{'winter_temp': 2.710003038671631, 'mortality'...",2021-04-11,"{'eggs': 1, 'non_foraging_days': 19, 'precipit...",...,2022-05-12,"{'eggs': 11, 'non_foraging_days': 9, 'precipit...","{'egg_mortality': 0.02727272727272728, 'cold_d...","{'mortality': 0.14000000000000007, 'dev_days':...","{'winter_temp': 3.4424062657433785, 'mortality...",2023-04-23,"{'eggs': 4, 'non_foraging_days': 16, 'precipit...","{'egg_mortality': 0.22500000000000003, 'cold_d...","{'mortality': 0.14750000000000008, 'dev_days':...","{'winter_temp': 3.3847727777193417, 'mortality..."
2,1085,184,1085_184,2020-05-24,"{'eggs': 13, 'non_foraging_days': 7, 'precipit...","{'egg_mortality': 0.0, 'cold_days': 0.0, 'hot_...","{'mortality': 0.15000000000000008, 'dev_days':...","{'winter_temp': 2.674645267959152, 'mortality'...",2021-04-11,"{'eggs': 1, 'non_foraging_days': 19, 'precipit...",...,2022-05-11,"{'eggs': 12, 'non_foraging_days': 8, 'precipit...","{'egg_mortality': 0.03333333333333333, 'cold_d...","{'mortality': 0.14000000000000007, 'dev_days':...","{'winter_temp': 3.4608536882681302, 'mortality...",2023-04-23,"{'eggs': 6, 'non_foraging_days': 14, 'precipit...","{'egg_mortality': 0.21666666666666667, 'cold_d...","{'mortality': 0.14750000000000008, 'dev_days':...","{'winter_temp': 3.2691770823793895, 'mortality..."
3,1086,184,1086_184,2020-05-23,"{'eggs': 14, 'non_foraging_days': 6, 'precipit...","{'egg_mortality': 0.0, 'cold_days': 0.0, 'hot_...","{'mortality': 0.14500000000000007, 'dev_days':...","{'winter_temp': 2.557405505121123, 'mortality'...",2021-04-11,"{'eggs': 1, 'non_foraging_days': 19, 'precipit...",...,2022-05-11,"{'eggs': 13, 'non_foraging_days': 7, 'precipit...","{'egg_mortality': 0.03076923076923077, 'cold_d...","{'mortality': 0.14000000000000007, 'dev_days':...","{'winter_temp': 3.405440408429614, 'mortality'...",2023-04-23,"{'eggs': 6, 'non_foraging_days': 14, 'precipit...","{'egg_mortality': 0.16666666666666666, 'cold_d...","{'mortality': 0.14750000000000008, 'dev_days':...","{'winter_temp': 3.0572748937360616, 'mortality..."
4,1081,185,1081_185,2020-05-25,"{'eggs': 13, 'non_foraging_days': 7, 'precipit...","{'egg_mortality': 0.015384615384615385, 'cold_...","{'mortality': 0.15000000000000008, 'dev_days':...","{'winter_temp': 2.745083035388962, 'mortality'...",2021-04-10,"{'eggs': 1, 'non_foraging_days': 19, 'precipit...",...,2022-05-11,"{'eggs': 10, 'non_foraging_days': 10, 'precipi...","{'egg_mortality': 0.04, 'cold_days': 0.4, 'hot...","{'mortality': 0.14000000000000007, 'dev_days':...","{'winter_temp': 3.327987202028662, 'mortality'...",2023-04-23,"{'eggs': 5, 'non_foraging_days': 15, 'precipit...","{'egg_mortality': 0.24000000000000005, 'cold_d...","{'mortality

In [26]:
grids.to_csv('/Users/edwardamoah/Documents/GitHub/OsmiaPopModel/osmiaPopModel_python/output/emergence_reproduction_13_9C_5mm_adam_November_2016_2019_all_cyle.csv', index=False)

In [ ]:
######### spatially explicit simulation #########

In [ ]:
### spatially explicit model

'''
grids_sp = forage[["col", "row", "grid_id"]]

for year in years:
    grids_sp[f"emergence_{year}"] = grids_sp.apply(lambda x: getEmergence(x["col"], x["row"], year), axis=1)
    grids_sp[f"reproduction_{year}"] = grids_sp.apply(lambda x: getSpatiallyExplicitReproduction(x["col"], x["row"], x[f"emergence_{year}"]), axis=1)
'''

'\ngrids_sp = forage[["col", "row", "grid_id"]]\n\nfor year in years:\n    grids_sp[f"emergence_{year}"] = grids_sp.apply(lambda x: getEmergence(x["col"], x["row"], year), axis=1)\n    grids_sp[f"reproduction_{year}"] = grids_sp.apply(lambda x: getSpatiallyExplicitReproduction(x["col"], x["row"], x[f"emergence_{year}"]), axis=1)\n'

In [ ]:
#grids_sp.to_csv('/Users/edwardamoah/Documents/GitHub/OsmiaPopModel/osmiaPopModel_python/output/spatially_explicit_reproduction_with_max_eggs_with_foraging_quality_5mm_precip_threshold_with_temp_and_forage_scores.csv', index=False)

In [ ]:
############ Study Sites ################

In [ ]:
'''
sites = ['GreenHouse', "Natalie", 'Mendel', 'Lemont']
lat = [40.802, 40.799, 40.884, 40.808]
lon = [-77.862, -77.905, -77.834, -77.82]

df_sites = pd.DataFrame({
    "site": sites,
    "lat": lat,
    "lon": lon
})


df_st = pd.DataFrame({
    "col": [1131, 1131, 1131, 1131, 1131, 1131, 1131, 1131, 1131, 1131, 1131, 1131, 1131, 1131, 1131, 1131],
    "row": [219, 219, 219, 219, 219, 219, 219, 219, 219, 219, 219, 219, 219, 219, 219, 219],
    "year": [2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]
})

df_st["emergence"] = df_st.apply(lambda x: getEmergence(x["col"], x["row"], x["year"]), axis=1)


#df_st["emergence_date"] = pd.to_datetime(df_st["emergence_date"])
#for year in years:
df_st["reproduction"] = df_st.apply(lambda x: getSpatiallyExplicitReproduction(x["col"], x["row"], x[f"emergence"]), axis=1)


df_st["non_foraging_days"] = [x['non_foraging_days'] for x in df_st["reproduction"].tolist()]
df_st["eggs"] = [x['eggs'] for x in df_st["reproduction"].tolist()]
df_st["precipitation_effect"] = [x['precipitation_effect'] for x in df_st["reproduction"].tolist()]
df_st["temperature_effect"] = [x['temperature_effect'] for x in df_st["reproduction"].tolist()]


def getJulianDay(emergence):
    emergence = datetime.strptime(emergence, '%Y-%m-%d').date()
    starting_date = datetime.strptime(f'{emergence.year}-01-01', '%Y-%m-%d').date()
    return (emergence - starting_date).days + 1


df_st["Julian_day"] = df_st["emergence"].apply(lambda x: getJulianDay(str(x)))
'''

'\nsites = [\'GreenHouse\', "Natalie", \'Mendel\', \'Lemont\']\nlat = [40.802, 40.799, 40.884, 40.808]\nlon = [-77.862, -77.905, -77.834, -77.82]\n\ndf_sites = pd.DataFrame({\n    "site": sites,\n    "lat": lat,\n    "lon": lon\n})\n\n\ndf_st = pd.DataFrame({\n    "col": [1131, 1131, 1131, 1131, 1131, 1131, 1131, 1131, 1131, 1131, 1131, 1131, 1131, 1131, 1131, 1131],\n    "row": [219, 219, 219, 219, 219, 219, 219, 219, 219, 219, 219, 219, 219, 219, 219, 219],\n    "year": [2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]\n})\n\ndf_st["emergence"] = df_st.apply(lambda x: getEmergence(x["col"], x["row"], x["year"]), axis=1)\n\n\n#df_st["emergence_date"] = pd.to_datetime(df_st["emergence_date"])\n#for year in years:\ndf_st["reproduction"] = df_st.apply(lambda x: getSpatiallyExplicitReproduction(x["col"], x["row"], x[f"emergence"]), axis=1)\n\n\ndf_st["non_foraging_days"] = [x[\'non_foraging_days\'] for x in df_st["reproduction"].tolist()]\ndf

In [ ]:
'''
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Sample DataFrame (replace this with your actual DataFrame)
data = {
    'x': [1, 2, 3, 4, 5],
    'y': [2, 3, 5, 4, 6]
}
df = df_st #pd.DataFrame(data)

columns = ['year', 'non_foraging_days', 'eggs', 'precipitation_effect', 'temperature_effect']
# Scatter plot
i=4
plt.scatter(df['year'], df[columns[i]], label='Number of Cold Days')

# Linear regression
coefficients = np.polyfit(df['year'], df[columns[i]], 1)
slope = coefficients[0]
intercept = coefficients[1]
trendline = f'Regression line: y={slope:.2f}x{intercept:.2f}'
plt.plot(df['year'], intercept + slope * df['year'], color='red', label=trendline)

# Add labels and legend
plt.xlabel('Year')
plt.ylabel('Days')
plt.title('Number of Cold (<13.9 C) Days vs Year')
plt.legend()

# Show plot
plt.show()
'''

"\nimport pandas as pd\nimport matplotlib.pyplot as plt\nimport numpy as np\n\n# Sample DataFrame (replace this with your actual DataFrame)\ndata = {\n    'x': [1, 2, 3, 4, 5],\n    'y': [2, 3, 5, 4, 6]\n}\ndf = df_st #pd.DataFrame(data)\n\ncolumns = ['year', 'non_foraging_days', 'eggs', 'precipitation_effect', 'temperature_effect']\n# Scatter plot\ni=4\nplt.scatter(df['year'], df[columns[i]], label='Number of Cold Days')\n\n# Linear regression\ncoefficients = np.polyfit(df['year'], df[columns[i]], 1)\nslope = coefficients[0]\nintercept = coefficients[1]\ntrendline = f'Regression line: y={slope:.2f}x{intercept:.2f}'\nplt.plot(df['year'], intercept + slope * df['year'], color='red', label=trendline)\n\n# Add labels and legend\nplt.xlabel('Year')\nplt.ylabel('Days')\nplt.title('Number of Cold (<13.9 C) Days vs Year')\nplt.legend()\n\n# Show plot\nplt.show()\n"

In [ ]:
'''
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df_st.plot(x='Julian_day', y='temperature_effect', kind='scatter', label='Emergence date')

# Linear regression
coefficients = np.polyfit(df_st['Julian_day'], df_st['temperature_effect'], 1)
slope = coefficients[0]
intercept = coefficients[1]
trendline = f'Regression line: y={slope:.2f}x+{intercept:.2f}'
plt.plot(df_st['Julian_day'], intercept + slope * df_st['Julian_day'], color='red', label=trendline)

# Add labels and legend
plt.xlabel('Julian Date of Emergence')
plt.ylabel('Number of Cold Days')
plt.title('Number of Cold (<13.9 C) Days vs Julian Date of Emergence')
plt.legend()

# Show plot
plt.show()
'''


"\nimport pandas as pd\nimport numpy as np\nimport matplotlib.pyplot as plt\n\ndf_st.plot(x='Julian_day', y='temperature_effect', kind='scatter', label='Emergence date')\n\n# Linear regression\ncoefficients = np.polyfit(df_st['Julian_day'], df_st['temperature_effect'], 1)\nslope = coefficients[0]\nintercept = coefficients[1]\ntrendline = f'Regression line: y={slope:.2f}x+{intercept:.2f}'\nplt.plot(df_st['Julian_day'], intercept + slope * df_st['Julian_day'], color='red', label=trendline)\n\n# Add labels and legend\nplt.xlabel('Julian Date of Emergence')\nplt.ylabel('Number of Cold Days')\nplt.title('Number of Cold (<13.9 C) Days vs Julian Date of Emergence')\nplt.legend()\n\n# Show plot\nplt.show()\n"